In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Lets check your GPU
if tf.config.list_logical_devices('GPU') != []:
    print("LET'S DO THISS!!! :DDDD")
else: 
    print("Oh dude... D:")

LET'S DO THISS!!! :DDDD


In [2]:
with open('data/input.txt') as f:
    content = f.read().lower()
    
print(content[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [3]:
"".join(sorted(set(content.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [21]:
text_vec_layer = keras.layers.TextVectorization(
                                                split="character",
                                                standardize="lower"
                                            )
text_vec_layer.adapt([content])
encoded = text_vec_layer([content])[0]

In [22]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), subtraindo 2 (pad e unknown)
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [23]:
n_tokens, dataset_size

(39, 1115394)

In [24]:
window_size = 128
batch_size = 32
ds = tf.data.Dataset.from_tensor_slices(encoded) # Fatia o ds em janelas de tamanho window_size
ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
ds = ds.shuffle(100_000, seed=42)
ds = ds.batch(batch_size)
ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [8]:
for X_batch, y_batch in ds.take(1):
    print(X_batch.shape, y_batch.shape)
    print(X_batch[0], y_batch[0])

(32, 128) (32, 128)
tf.Tensor(
[ 9 12  0 19  5 20  6  2  0 16  5  2  6  0  2 27  3  2  6  1  8 17 10  1
  8  1  0  7  2  4 15  0 21  1  6  5  9 12  0  2  6  5  7  0 21 13  7  5
  9  1  7  7 26 10 10 14  1  9  1  9  5 13  7 23 10  3 17  0  2  8 13  1
 31 21  8  1 12 30 10 10 19  5  8  7  2  0  7  1  9  4  2  3  8 23 10 15
  3 13  8  0 18  3 14 22  4  9 15  0  2  3  0  2  6  1  0 18  4 22  5  2
  3 11 28  0 16  6  1  8], shape=(128,), dtype=int64) tf.Tensor(
[12  0 19  5 20  6  2  0 16  5  2  6  0  2 27  3  2  6  1  8 17 10  1  8
  1  0  7  2  4 15  0 21  1  6  5  9 12  0  2  6  5  7  0 21 13  7  5  9
  1  7  7 26 10 10 14  1  9  1  9  5 13  7 23 10  3 17  0  2  8 13  1 31
 21  8  1 12 30 10 10 19  5  8  7  2  0  7  1  9  4  2  3  8 23 10 15  3
 13  8  0 18  3 14 22  4  9 15  0  2  3  0  2  6  1  0 18  4 22  5  2  3
 11 28  0 16  6  1  8  1], shape=(128,), dtype=int64)


In [25]:
# Criando função
def to_dataset(sequence, window_size=128, batch_size=32, seed=42, shuffle=False, target='all_window'):
    ds = tf.data.Dataset.from_tensor_slices(sequence) # Fatia o ds em janelas de tamanho window_size
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    if target == 'all_window':
        ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)
    elif target == 'last_char':
        ds = ds.map(lambda window: (window[:, :-1], window[:, -1])).prefetch(1)
    else:
        raise ValueError('target must be "all_window" or "last_char"')
    return ds

In [26]:
# Exemplo de uso
list(to_dataset(text_vec_layer(['To be'])[0], window_size=3))

[(<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 4,  5,  2],
         [ 5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 5,  2, 23],
         [ 2, 23,  3]], dtype=int64)>)]

In [27]:
window_size = 128
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], window_size=window_size, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], window_size=window_size)
test_set = to_dataset(encoded[1_060_000:], window_size=window_size)

### Training Char-RNN model

In [31]:
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self, n_tokens):
        super(CustomModel, self).__init__()
        self.embedding = keras.layers.Embedding(input_dim=n_tokens, output_dim=16, input_shape=[None, 128])
        self.conv1d = keras.layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu")
        self.gru = keras.layers.GRU(128, return_sequences=True)
        # Adicionando uma camada de atenção
        self.attention = keras.layers.Attention()
        self.dense = keras.layers.Dense(n_tokens, activation="softmax")
        
    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        x = self.conv1d(x)
        x = self.gru(x)
        # Aplicando atenção
        # A camada de atenção precisa de duas entradas, neste caso, usamos `x` duas vezes.
        x = self.attention([x, x])
        x = self.dense(x)
        return x
    
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Configurando a seed para reprodutibilidade
tf.random.set_seed(42)

model = CustomModel(n_tokens=n_tokens)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)


In [13]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch, y_batch)

tf.Tensor(
[[15  0  9 ... 15 30 10]
 [25  1  0 ...  0  3 19]
 [13  7 23 ...  5  9  0]
 ...
 [25  1  0 ...  7 23 10]
 [11 12  0 ...  6  1  0]
 [ 1  0  5 ...  2  1  8]], shape=(32, 128), dtype=int64) tf.Tensor(
[[ 0  9  3 ... 30 10 10]
 [ 1  0  3 ...  3 19  0]
 [ 7 23 10 ...  9  0  8]
 ...
 [ 1  0 21 ... 23 10 15]
 [12  0 15 ...  1  0  7]
 [ 0  5  2 ...  1  8 27]], shape=(32, 128), dtype=int64)


In [14]:
X_batch.shape

TensorShape([32, 128])

In [33]:
model(X_batch)

<tf.Tensor: shape=(32, 128, 100), dtype=float32, numpy=
array([[[0.00995005, 0.00998792, 0.00998975, ..., 0.00986534,
         0.00993769, 0.0100949 ],
        [0.00995005, 0.00998791, 0.00998975, ..., 0.00986535,
         0.00993769, 0.01009491],
        [0.00995005, 0.00998792, 0.00998974, ..., 0.00986535,
         0.00993769, 0.01009492],
        ...,
        [0.00995005, 0.00998793, 0.00998975, ..., 0.00986534,
         0.00993769, 0.0100949 ],
        [0.00995004, 0.00998793, 0.00998976, ..., 0.00986534,
         0.0099377 , 0.01009492],
        [0.00995005, 0.00998793, 0.00998976, ..., 0.00986535,
         0.0099377 , 0.01009492]],

       [[0.00997565, 0.00998185, 0.00999155, ..., 0.00987234,
         0.00994663, 0.01008519],
        [0.00997565, 0.00998185, 0.00999155, ..., 0.00987237,
         0.00994665, 0.01008518],
        [0.00997566, 0.00998185, 0.00999155, ..., 0.00987235,
         0.00994665, 0.01008518],
        ...,
        [0.00997564, 0.00998185, 0.00999154, ..., 0.

In [34]:
model.summary()

Model: "custom_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  1600      
                                                                 
 conv1d_3 (Conv1D)           multiple                  1568      
                                                                 
 gru_3 (GRU)                 multiple                  62208     
                                                                 
 attention_3 (Attention)     multiple                  0         
                                                                 
 dense_3 (Dense)             multiple                  12900     
                                                                 
Total params: 78,276
Trainable params: 78,276
Non-trainable params: 0
_________________________________________________________________


In [35]:
# estilo
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = '#26262e' 

def print_multiple_generated(predicted_text, original_text, n=8):
    print("--> Original text:")
    for i in range(n):
        print(original_text[i])
    print("\n--> Generated text:")
    for i in range(n):
        print(predicted_text[i])
        
def plot_loss(history, step):
    plt.plot(list(range(len(history['loss']))), history['loss'], label='Training Loss')
    plt.plot(list(range(len(history['loss']))), history['val_loss'], label='Validation Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_accuracy(history, step):
    plt.plot(list(range(len(history['accuracy']))), history['accuracy'], label='Training Accuracy')
    plt.plot(list(range(len(history['accuracy']))), history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Step')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [36]:
history = {
            'text_base': 'To be or not ',
            'text_base_encoded': text_vec_layer(['To be or not ']),
            'list_predicted_text_base': [],
        }

class LogCallback(tf.keras.callbacks.Callback):
    def __init__(self, history):
        super().__init__()
        self.history_ = history
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"Final da época {epoch}, Loss: {logs['loss']}, Acc: {logs['accuracy']}")
    
    def on_train_batch_end(self, batch, logs=None):
        if batch % 1_000 == 0:
            print("")
            print(f"Final do lote {batch}, Loss do lote: {logs['loss']}")
            model = self.model
            # predicted_text = decoder(tf.argmax(model(history['text_base_encoded'], training=False), axis=2))[0]
            predicted_text = extend_text("To be or not ", temperature=1.0)
            print("")
            print("--> Original text: ", history['text_base'])
            print("--> Generated text: ", predicted_text)
            self.history_['list_predicted_text_base'].append(predicted_text)
            print("")

In [44]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_custom_model_w_attention", monitor="val_accuracy", save_best_only=True)

shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text], verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    print(char_id)
    return text_vec_layer.get_vocabulary()[char_id]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [45]:
len(text_vec_layer.get_vocabulary())

41

In [46]:
history_train = model.fit(train_set, 
                    validation_data=valid_set, 
                    epochs=10,
                    callbacks=[LogCallback(history=history), 
                               model_ckpt])

Epoch 1/10

Final do lote 0, Loss do lote: 4.591318130493164
tf.Tensor(52, shape=(), dtype=int64)


IndexError: list index out of range